In [1]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [2]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [3]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REGISTRATION_NO', 'des_col': 'registration_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSUMER_ID', 'des_col': 'consumer_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_NAME', 'des_col': 'complainant_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REQUESTNATURE_ID', 'des_col': 'requestnature_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_DOOR_NO', 'des_col': 'complainant_door_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_STREET', 'des_col': 'complainant_street', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_AREA', 'des_col': 'complainant_area', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_CITY', 'des_col': 'complainant_city', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_PINCODE', 'des_col': 'complainant_pincode', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_LAND_MARKS', 'des_col': 'complainant_land_marks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_PHONE', 'des_col': 'complainant_phone', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_FAX', 'des_col': 'complainant_fax', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'COMPLAINANT_EMAIL', 'des_col': 'complainant_email', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REQUEST_DETAILS', 'des_col': 'request_details', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PRIORITY', 'des_col': 'priority', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'ORIGIN', 'des_col': 'origin', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'STATUS', 'des_col': 'status', 'src_dtype': 'DECFLOAT', 'des_dtype': 'integer'},
    {'src_col': 'WAIT_FOR_PAYMENT', 'des_col': 'wait_for_payment', 'character': 'String', 'des_dtype': 'character'},
    {'src_col': 'SERVICE_ORDER_NO', 'des_col': 'service_order_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALLOCATED_TO', 'des_col': 'allocated_to', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALLOCATED_ON', 'des_col': 'allocated_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'NEXT_FOLLOW_UP_TIME', 'des_col': 'next_follow_up_time', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'TO_BE_RECTIFIED_ON', 'des_col': 'to_be_rectified_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'FIRST_ESCALATION_ON', 'des_col': 'first_escalation_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'FIRST_ESCALATIONTO_ID', 'des_col': 'first_escalationto_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SECOND_ESCALATION_ON', 'des_col': 'second_escalation_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'SECOND_ESCALATIONTO_ID', 'des_col': 'second_escalationto_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RESPONSIBLEPERSON_ID', 'des_col': 'responsibleperson_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESCALATION_DETAILS', 'des_col': 'escalation_details', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PR_NO', 'des_col': 'pr_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECTIFIED_ON', 'des_col': 'rectified_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'RECTIFIED_BY', 'des_col': 'rectified_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REQUEST_REGISTERED_ON', 'des_col': 'request_registered_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'REQUEST_REGISTERED_BY', 'des_col': 'request_registered_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACKNOWLEDGED_BY', 'des_col': 'acknowledged_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACKNOWLEDGED_ON', 'des_col': 'acknowledged_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'IS_ON_HOLD', 'des_col': 'is_on_hold', 'src_dtype': 'character', 'des_dtype': 'boolean'},
    {'src_col': 'REASON_FOR_HOLD', 'des_col': 'reason_for_hold', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACTION_TAKEN', 'des_col': 'action_taken', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSUMER_REMARKS', 'des_col': 'consumer_remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TO_BE_INSPECTED_ON', 'des_col': 'to_be_inspected_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'ACTUAL_INSPECTED_DT', 'des_col': 'actual_inspected_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'FORWARDED_ON', 'des_col': 'forwarded_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'PAYMENT_MODE', 'des_col': 'payment_mode', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PAYMENT_AMOUNT', 'des_col': 'payment_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PAYMENT_DUE_DATE', 'des_col': 'payment_due_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'REASON_FOR_PAYMENT', 'des_col': 'reason_for_payment', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PAYMENT_MADE_ON', 'des_col': 'payment_made_on', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'REASON_FOR_PENDING', 'des_col': 'reason_for_pending', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACTUAL_REC_DATE', 'des_col': 'actual_rec_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'ACTUAL_INV_DATE', 'des_col': 'actual_inv_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'REFERENCE_NO', 'des_col': 'reference_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APP_FEE', 'des_col': 'app_fee', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'REASON_MODIFICATION', 'des_col': 'reason_modification', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISPATCH_LETTER_NO', 'des_col': 'dispatch_letter_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NO_OF_HOLIDAYS', 'des_col': 'no_of_holidays', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'REFUND_AMT', 'des_col': 'refund_amt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LOAD_CHANGE', 'des_col': 'load_change', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'UNIT_CHANGE', 'des_col': 'unit_change', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SC_CH', 'des_col': 'sc_ch', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'OTH_CH', 'des_col': 'oth_ch', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CONSUMER_CHANGES', 'des_col': 'consumer_changes', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RESPONSIBLEPERSON_SHOW', 'des_col': 'responsibleperson_show', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APPROVED_PER', 'des_col': 'approved_per', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REFUND_DD', 'des_col': 'refund_dd', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REFUND_DATE', 'des_col': 'refund_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'COMPLAINANT_VILLAGE', 'des_col': 'complainant_village', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_METER_CHANGE_REQ', 'des_col': 'is_meter_change_req', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_CHANGE_PHASE', 'des_col': 'meter_change_phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_CONS_PAY_REQ', 'des_col': 'is_cons_pay_req', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PAY_TYPE', 'des_col': 'pay_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METER_TYPE', 'des_col': 'meter_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FAULT_TYPE', 'des_col': 'fault_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FEASIBILITY', 'des_col': 'feasibility', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DISPATCH_FLAG', 'des_col': 'dispatch_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FINAL_READING', 'des_col': 'final_reading', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SETTLEMENT_AMT', 'des_col': 'settlement_amt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'SETTLEMENTOFFICE_ID', 'des_col': 'settlementoffice_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REGESTERED_AT', 'des_col': 'regestered_at', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SETTLEMENT_DT', 'des_col': 'settlement_dt', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'SETTLED_AMOUNT', 'des_col': 'settled_amount', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DECISION', 'des_col': 'decision', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONSUMER_SATIFICATION', 'des_col': 'consumer_satification', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CCAR_FLAG', 'des_col': 'ccar_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECONN_CHG', 'des_col': 'reconn_chg', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'FIX_CHG', 'des_col': 'fix_chg', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'APPLICANT_SIGN', 'des_col': 'applicant_sign', 'src_dtype': 'BLOB', 'des_dtype': 'String'},
    {'src_col': 'RAPIDLOC', 'des_col': 'rapidloc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RAPIDFAULT', 'des_col': 'rapidfault', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ITEM_CATEGORY', 'des_col': 'item_category', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ITEM_NAME', 'des_col': 'item_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADJSTMENT_STATUS', 'des_col': 'adjstment_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CSC_TRACKING_HISTORY', 'des_col': 'csc_tracking_history', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ESCALLATION_JSON', 'des_col': 'escallation_json', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACKNOWLEDGEMENT_ID', 'des_col': 'acknowledgement_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACCOUNTCLOSURE_DOC', 'des_col': 'accountclosure_doc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NAMECHANGE_DOC', 'des_col': 'namechange_doc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'METEREADING_DOC', 'des_col': 'metereading_doc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TAZKARI_DOC', 'des_col': 'tazkari_doc', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IDPROOF', 'des_col': 'idproof', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ACCOUNTBOOK', 'des_col': 'accountbook', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MTR_STATUS', 'des_col': 'mtr_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PROVINCE', 'des_col': 'province', 'src_dtype': 'String', 'des_dtype': 'String'}
    
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.CSTREQUEST_T', 'public', 'cstrequest_t', cols_map)
print("Total Migrated Records (cstrequest_t): " + str(migrated_row_count))


Total Migrated Records (cstrequest_t): Total Src Records: 48238 Total migrated Records to dest: 48238
